In [6]:
!pip install ultralytics

  Using cached ultralytics-8.3.78-py3-none-any.whl.metadata (35 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.78-py3-none-any.whl (921 kB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)


In [7]:
!pip install mediapipe

In [8]:
!pip install easyocr

In [4]:
# import os
# if not os.path.exists("deep_sort_realtime"):
!git clone https://github.com/levan92/deep_sort_realtime.git
!pip install -r deep_sort_realtime/requirements.txt

Cloning into 'deep_sort_realtime'...
remote: Enumerating objects: 885, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 885 (delta 177), reused 155 (delta 155), pack-reused 658 (from 1)
Receiving objects: 100% (885/885), 16.14 MiB | 28.99 MiB/s, done.
Resolving deltas: 100% (538/538), done.


In [ ]:
#Rename the root directory as "deep_sort_realtime0"

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!mv deep_sort_realtime0/* .
!rm -r deep_sort_realtime0

In [3]:
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
import os
import datetime
import time
import numpy as np
import pandas as pd
from collections import deque
from tqdm import tqdm
from PIL import Image

import cv2
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# ---------------------------
# Facial Emotion Classification (FER)
# ---------------------------
# Assuming your deep emotion model is defined in deep_emotion.Deep_Emotion
from deep_emotion import Deep_Emotion

FERMODEL_PATH = 'best_model1.pth'

def classify_image(image_array):
    """
    Classify emotion from an image using your Deep_Emotion model.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Deep_Emotion()
    model.load_state_dict(torch.load(FERMODEL_PATH, map_location=device))
    model.to(device)
    model.eval()

    transformation = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    image = Image.fromarray(image_array).convert('L')
    image = transformation(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(F.softmax(output, dim=1), 1).item()

    classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    return classes[predicted_class]
import datetime

def classify_emotion(face_image):
    """
    Classify emotion using the above classifier.
    Only consider negative emotions; otherwise return Neutral.
    """
    emotion = classify_image(face_image)
    return emotion if emotion in ["Sad", "Fear", "Disgust", "Angry"] else "Neutral"

# ---------------------------
# Report Writing Function
# ---------------------------
def write_report(video_path, output_video_path, total_frames, male_count, female_count,men,women,
                 lone_women_instances, sos_count, track_license_plates,report_path):
    """Generates and saves a detailed report of the video analysis."""
    # Compute gender ratio (M/F)
    if women == 0:
        gender_ratio = "Only males detected"
    else:
        gender_ratio = f"{men // women}:1" if male_count >= female_count else f"1:{women // men}"

    # Compute unique number plates
    unique_plates = len(set(track_license_plates.values()))
    avg_plates_per_frame = unique_plates / total_frames if total_frames else 0

    analysis_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    report_dir = os.path.dirname(report_path)  # Get the directory part of the path
    if not os.path.exists(report_dir):
        os.makedirs(report_dir)

    # report_path = "NEW_NEW_report.txt"
    with open(report_path, "w") as f:
        f.write("Video Analysis Report\n")
        f.write("===================\n\n")
        f.write(f"Analysis performed on: {analysis_time}\n")
        f.write(f"Input video: {video_path}\n")
        f.write(f"Output video: {output_video_path}\n\n")
        f.write("Statistics:\n")
        f.write(f"- Total frames processed: {total_frames}\n")
        # f.write(f"- Total males detected: {male_count}\n")
        # f.write(f"- Total females detected: {female_count}\n")
        f.write(f"- Finally Total Men detected: {men}\n")
        f.write(f"- Finally Total Women detected: {women}\n")
        f.write(f"- Gender ratio (M/F): {gender_ratio}\n")
        f.write(f"- Lone women detected (tracking IDs): {len(lone_women_instances)}\n")
        f.write(f"- Total SOS instances detected: {sos_count}\n")
        f.write(f"- Unique number plates: {unique_plates}\n")
        f.write(f"- Average plates per frame: {avg_plates_per_frame:.2f}\n\n")
        if lone_women_instances:
            f.write("Lone Women Detected (tracking IDs):\n")
            for tid in lone_women_instances:
                f.write(f"- ID: {tid}\n")
    print(f"Detection completed. Report saved to `{report_path}`.")

# ---------------------------
# Dummy ANPR Function (Replace with your implementation)
# ---------------------------
import cv2
import easyocr
from ultralytics import YOLO

# Load license plate detection model
license_plate_detector = YOLO("license_plate_detector.pt")

# Initialize OCR reader
reader = easyocr.Reader(['en'], gpu=True)

def read_license_plate(license_plate_crop):
    """Reads the license plate text using EasyOCR."""
    detections = reader.readtext(license_plate_crop)
    for detection in detections:
        _, text, score = detection
        text = text.upper().replace(' ', '')
        return text if len(text) > 3 else None  # Avoid false positives
    return None

def perform_anpr(frame, car_bbox):
    """
    Detects the license plate inside the car and extracts the number.
    """
    x1, y1, x2, y2 = car_bbox
    car_crop = frame[y1:y2, x1:x2]

    if car_crop.size == 0 or car_crop.shape[0] == 0 or car_crop.shape[1] == 0:
        return None  # Skip ANPR if the crop is invalid

    license_plates = license_plate_detector(car_crop)[0]
    for license_plate in license_plates.boxes.data.tolist():
        lx1, ly1, lx2, ly2, score, _ = license_plate
        lx1, ly1, lx2, ly2 = map(int, [lx1, ly1, lx2, ly2])
        license_plate_crop = car_crop[ly1:ly2, lx1:lx2]
        license_text = read_license_plate(license_plate_crop)
        if license_text:
            return license_text
    return None


# ---------------------------
# Initialize YOLO, DeepSORT, and Models
# ---------------------------
# DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3, nn_budget=10)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load gender classification model
loaded_model = torch.load('gender_classification_model.pth', map_location=torch.device('cpu'))
loaded_model.eval()

# YOLO models for object and face detection
model = YOLO('yolov8n.pt')   # Object detection model
face_model = YOLO('best.pt') # Face detection model

# ---------------------------
# Utility Functions for Gender Classification
# ---------------------------
def preprocess_image(image):
    """Preprocess image before feeding to gender classification model."""
    image = cv2.resize(image, (224, 224))
    image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
    return image

def classify_gender(cropped_image):
    """Classify gender based on the trained model."""
    with torch.no_grad():
        image_tensor = preprocess_image(cropped_image)
        output = loaded_model(image_tensor)
        _, predicted_class = torch.max(output, 1)
        return "Man" if predicted_class.item() == 0 else "Woman"

# ---------------------------
# Lone Women Tracking
# ---------------------------
lone_women_tracker = {}  # key: tracking ID, value: dict with start_time, duration, confirmed
LONE_WOMAN_THRESHOLD = 10  # seconds

def update_lone_women(detected_women, current_timestamp):
    """
    Update the tracking of lone women.
    A woman is confirmed as "lone" if she remains the only detected woman for at least LONE_WOMAN_THRESHOLD seconds.
    """
    global lone_women_tracker
    active_ids = set(lone_women_tracker.keys())
    detected_set = set(detected_women)

    # For IDs no longer detected, check if they reached the threshold
    for woman_id in list(active_ids - detected_set):
        if lone_women_tracker[woman_id]['duration'] >= LONE_WOMAN_THRESHOLD:
            lone_women_tracker[woman_id]['confirmed'] = True
        else:
            del lone_women_tracker[woman_id]

    # Update or add currently detected women
    for woman_id in detected_set:
        if woman_id not in lone_women_tracker:
            lone_women_tracker[woman_id] = {'start_time': current_timestamp, 'duration': 0, 'confirmed': False}
        else:
            lone_women_tracker[woman_id]['duration'] = current_timestamp - lone_women_tracker[woman_id]['start_time']

# ---------------------------
# SOS Detection using Mediapipe
# ---------------------------
if not os.path.exists("SOS_Logs"):
    os.makedirs("SOS_Logs")

def log_sos_event(frame):
    """Saves an SOS frame and logs the timestamp."""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"SOS_Logs/SOS_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"[ALERT] SOS Detected! Frame saved: {filename}")
    with open("SOS_Logs/sos_log.txt", "a") as log_file:
        log_file.write(f"{timestamp} - SOS Detected!\n")

import mediapipe as mp
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

person_motion = deque(maxlen=10)  # For tracking movement

def detect_movement(pose_landmarks, frame_width, frame_height):
    """Detects running and sudden movements."""
    global person_motion
    key_points = [pose_landmarks.landmark[i] for i in [mp_pose.PoseLandmark.LEFT_HIP,
                                                       mp_pose.PoseLandmark.RIGHT_HIP,
                                                       mp_pose.PoseLandmark.LEFT_KNEE,
                                                       mp_pose.PoseLandmark.RIGHT_KNEE]]
    key_positions = [(int(p.x * frame_width), int(p.y * frame_height)) for p in key_points]
    avg_x = sum(p[0] for p in key_positions) // len(key_positions)
    avg_y = sum(p[1] for p in key_positions) // len(key_positions)
    person_motion.append((avg_x, avg_y))
    if len(person_motion) > 5:
        deltas = np.diff(person_motion, axis=0)
        speeds = np.sum(np.abs(deltas), axis=0)
        if speeds[1] > 50:
            return "Running"
        if np.linalg.norm(deltas[-1]) > 40:
            return "Sudden Movement"
    return "Normal"

def detect_hand_gesture(hand_landmarks, frame_width, frame_height):
    """Detects raised hand gesture."""
    landmarks = [hand_landmarks.landmark[i] for i in range(21)]
    y_positions = [l.y for l in landmarks]
    avg_y = sum(y_positions) / len(y_positions)
    hand_height = avg_y * frame_height
    if hand_height < frame_height // 3:
        return "Raised Hand"
    return "Normal"

SOS_cooldown = {}  # To prevent multiple triggers for the same track
SOS_THRESHOLD = 3   # Number of consecutive frames needed to trigger SOS

def perform_sos(person, track_id, frame_number):
    """
    Detects SOS condition based on facial emotion, body movement, and hand gesture.
    Returns "SOS Detected" if at least two of the conditions are triggered; otherwise "Safe".
    """
    global SOS_cooldown
    if track_id in SOS_cooldown and frame_number - SOS_cooldown[track_id] < SOS_THRESHOLD*2:
        return "Safe"  # within cooldown period

    # Check for valid crop; skip if it's too small
    min_dim = 20  # set a minimum acceptable dimension
    if person is None or person.size == 0 or person.shape[0] < min_dim or person.shape[1] < min_dim:
        # Optionally, you could log this once per track if needed.
        return "Safe"

    rgb_person = cv2.cvtColor(person, cv2.COLOR_BGR2RGB)
    pose_results = pose.process(rgb_person)
    hand_results = hands.process(rgb_person)

    emotion = classify_emotion(person)
    movement = "Normal"
    hand_gesture = "Normal"

    if pose_results.pose_landmarks:
        movement = detect_movement(pose_results.pose_landmarks, person.shape[1], person.shape[0])
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            hand_gesture = detect_hand_gesture(hand_landmarks, person.shape[1], person.shape[0])

    triggers = sum([
        emotion in ["Sad", "Fear", "Disgust", "Angry"],
        movement in ["Running", "Sudden Movement"],
        hand_gesture == "Raised Hand"
    ])

    if triggers >= 2 and frame_number - SOS_cooldown.get(track_id, 0) > SOS_THRESHOLD * 2:
        SOS_cooldown[track_id] = frame_number
        log_sos_event(person)
        return "SOS Detected"
    return "Safe"


def shrink_bbox(x1, y1, x2, y2, shrink_factor=0.2):
    """
    Shrinks the bounding box by the given shrink_factor (default 20%).
    Returns new coordinates.
    """
    w = x2 - x1
    h = y2 - y1
    dx = int(w * shrink_factor / 2)
    dy = int(h * shrink_factor / 2)
    return x1 + dx, y1 + dy, x2 - dx, y2 - dy

# ---------------------------
# Main Detection Function
# ---------------------------
def detect_objects(video_path, output_video_path,report_path,csv_path, conf=0.5):
    """
    Detects objects in a video, classifies gender,
    performs SOS detection, tracks lone women, and processes ANPR for cars.
    Draws bounding boxes: blue for Men, green for Women, red for SOS.
    Generates output video, report, CSV, and SOS logs.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video stream or file")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or 1
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    track_gender = {}
    track_license_plates = {}  # For cars
    cars_data = []  # To be saved as CSV
    male_count, female_count, sos_count = 0, 0, 0

    # Process each frame
    with tqdm(total=total_frames, desc="Processing Video", unit="frame") as pbar:
        frame_number = 0
        while True:
            ret, frame = cap.read()
            if not ret or frame_number>10:
                break
            frame_number += 1
            current_timestamp = frame_number / fps  # in seconds
            detected_women = []  # For lone women tracking in this frame

            # Run YOLO detection
            yolo_results = model.predict(source=frame, conf=conf, verbose=False)
            detections = []
            for result in yolo_results:
                boxes = result.boxes.cpu().numpy()
                for box in boxes:
                    b = box.xyxy[0].astype(int)
                    class_id = int(box.cls[0])
                    confidence = box.conf[0]
                    if confidence > conf and model.names[class_id] in ["person", "car"]:
                        detections.append([[b[0], b[1], b[2], b[3]], confidence, class_id])

            # Update tracks with DeepSORT
            tracks = tracker.update_tracks(detections, frame=frame)

            # Process each track
            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                x1, y1, x2, y2 = map(int, track.to_ltrb())
                class_id = track.det_class

                if model.names[class_id] == "person":
                    x1, y1, x2, y2 = map(int, track.to_ltrb())
                    x1 = max(0, x1)
                    y1 = max(0, y1)
                    x2 = min(frame.shape[1], x2)
                    y2 = min(frame.shape[0], y2)
                    if x2 <= x1 or y2 <= y1:
                        continue

                    cropped_person = frame[y1:y2, x1:x2]
                    if cropped_person.size == 0 or cropped_person.shape[0] < 20 or cropped_person.shape[1] < 20:
                        continue # Skip SOS detection if the crop is too small\n
                    if track_id not in track_gender:
                        if cropped_person.shape[0] > 0 and cropped_person.shape[1] > 0:
                            gender = classify_gender(cropped_person)
                            track_gender[track_id] = gender
                            if gender == "Man":
                                male_count += 1
                            elif gender == "Woman":
                                female_count += 1
                        else:
                            track_gender[track_id] = "Unknown"
                            gender = "Unknown"
                    else:
                        gender = track_gender[track_id]

                    if gender == "Man":
                        male_count += 1
                        # Draw blue bounding box for Men (BGR: Blue)
                        new_x1, new_y1, new_x2, new_y2 = shrink_bbox(x1, y1, x2, y2, shrink_factor=0.2)
                        cv2.rectangle(frame, (new_x1, new_y1), (new_x2, new_y2), (255, 0, 0), 2)
                        cv2.putText(frame, "Man", (new_x1, new_y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                    elif gender == "Woman":
                        female_count += 1
                        detected_women.append(track_id)
                        # Run face detection on the cropped person for further emotion analysis
                        if cropped_person.shape[0] > 0 and cropped_person.shape[1] > 0:
                            face_results = face_model.predict(source=cropped_person, conf=0.5, verbose=False)
                            if face_results and len(face_results[0].boxes) > 0:
                                # Optionally, use face detection to crop face region (if needed by your pipeline)
                                face_box = face_results[0].boxes[0].xyxy[0].int().tolist()
                                x1f, y1f, x2f, y2f = face_box
                                cropped_face = cropped_person[y1f:y2f, x1f:x2f]
                                # You can also use classify_emotion(cropped_face) here if desired.
                        # Check for SOS condition
                        sos_status = perform_sos(cropped_person, track_id, frame_number)
                        new_x1, new_y1, new_x2, new_y2 = shrink_bbox(x1, y1, x2, y2, shrink_factor=0.2)
                        if sos_status == "SOS Detected":
                            sos_count += 1
                            # Draw red bounding box for SOS
                            cv2.rectangle(frame, (new_x1, new_y1), (new_x2, new_y2), (0, 0, 255), 2)
                            cv2.putText(frame, "SOS Detected", (new_x1, new_y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        else:
                            # Draw green bounding box for Woman
                            cv2.rectangle(frame, (new_x1, new_y1), (new_x2, new_y2), (0, 255, 0), 2)
                            cv2.putText(frame, "Woman", (new_x1, new_y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                elif model.names[class_id] == "car":
                    if track_id not in track_license_plates:
                        license_plate_text = perform_anpr(frame, (x1, y1, x2, y2))
                        if license_plate_text:
                            track_license_plates[track_id] = license_plate_text
                            cars_data.append(["Car", track_id, None, license_plate_text])
            # Update lone women tracking based on detected women IDs in this frame
            update_lone_women(detected_women, current_timestamp)
            out.write(frame)
            pbar.update(1)

    cap.release()
    out.release()
    # mencounted = sum(1 for gender in track_gender.values() if gender == "Man")
    # womencounted = sum(1 for gender in track_gender.values() if gender == "Woman")
    men = sum(1 for gender in track_gender.values() if gender == "Man")
    women = sum(1 for gender in track_gender.values() if gender == "Woman")
    confirmed_lone_women = [k for k, v in lone_women_tracker.items() if v['confirmed']]
    write_report(video_path, output_video_path, total_frames, male_count, female_count,men,women,
                 confirmed_lone_women, sos_count, track_license_plates,report_path)

    # Save car ANPR results to CSV
    df = pd.DataFrame(cars_data, columns=["Object", "ID", "Gender", "License Plate"])
    df.to_csv(csv_path, index=False)
    print("Detection completed. Report, CSV, and output video have been generated.")

# ---------------------------
# Run the Detection Pipeline
# # ---------------------------
# if __name__ == "__main__":
#     detect_objects("/content/sample_video_ppl.mp4", "finally_final_output_video.mp4")

if __name__ == "__main__":
    detect_objects("/content/sample_vide_ppl.mp4", "output_video.mp4","1/Report.txt","1/Car_results.csv")


Processing Video:   1%|          | 4/621 [00:00<01:25,  7.19frame/s]/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4969: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Processing Video:   1%|          | 6/621 [00:02<05:07,  2.00frame/s]

[ALERT] SOS Detected! Frame saved: SOS_Logs/SOS_2025-02-19_18-00-12.jpg
[ALERT] SOS Detected! Frame saved: SOS_Logs/SOS_2025-02-19_18-00-12.jpg
[ALERT] SOS Detected! Frame saved: SOS_Logs/SOS_2025-02-19_18-00-12.jpg


Processing Video:   1%|          | 7/621 [00:02<05:05,  2.01frame/s]

[ALERT] SOS Detected! Frame saved: SOS_Logs/SOS_2025-02-19_18-00-12.jpg


Processing Video:   2%|▏         | 11/621 [00:03<03:21,  3.02frame/s]

Detection completed. Report saved to `1/Report.txt`.
Detection completed. Report, CSV, and output video have been generated.


In [ ]:
#Updated_version

In [9]:
import os
import datetime
import time
import numpy as np
import pandas as pd
from collections import deque
from tqdm import tqdm
from PIL import Image
import datetime
import easyocr
import torch.nn as nn
import cv2
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from deep_emotion import Deep_Emotion


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [45]:
FERMODEL_PATH = '/content/emotion_recog.pth'
license_plate_detector = YOLO("license_plate_detector.pt")

# Initialize OCR reader
reader = easyocr.Reader(['en'], gpu=True)

# DeepSORT tracker
tracker = DeepSort(max_age=120, n_init=3, nn_budget=10)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GenderClassificationModel(nn.Module):
    def __init__(self):
        super(GenderClassificationModel, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.model.classifier[1].in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load gender classification model
loaded_model = torch.load('gender_classification_full.pth', map_location=torch.device('cpu'))
loaded_model.eval()

# YOLO models for object and face detection
model = YOLO('yolov8n.pt')   # Object detection model
face_model = YOLO('/content/face_yolo.pt') # Face detection model

# ---------------------------
lone_women_tracker = {}  # key: tracking ID, value: dict with start_time, duration, confirmed
LONE_WOMAN_THRESHOLD = 10  # seconds

In [46]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to match EfficientNet input size
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

def predict_gender(image_array,model=loaded_model):
    image = Image.fromarray(image_array).convert("RGB")
    image = transform(image).unsqueeze(0)#.to(device)

    # Move both model and image to the same device
    model = model.to(device)
    image = image.to(device)

    with torch.no_grad():
        output = model(image)
        predicted = (output[0] > 0.5).float().item()

    return "Man" if predicted == 1 else "Woman"

# Example Usage
img='prabhas1.jpg'
image = Image.open(img).convert('RGB')
image_array = np.array(image)
prediction = predict_gender(image_array,loaded_model)
print(f"Predicted Gender: {prediction}")


# # Example Usage
# img=cv2.imread('prabhas1.jpg')
# # image = Image.open(img).convert('RGB') # This line was incorrect
# image = Image.fromarray(img).convert('RGB')  # Correct way to create an Image object from a NumPy array
# image_array = np.array(image)
# prediction = predict_gender(image_array,loaded_model)
# print(f"Predicted Gender: {prediction}")

Predicted Gender: Man


In [47]:
def classify_image(image_array):
    """
    Classify emotion from an image using your Deep_Emotion model.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Deep_Emotion()
    model.load_state_dict(torch.load(FERMODEL_PATH, map_location=device))
    model.to(device)
    model.eval()

    transformation = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    image = Image.fromarray(image_array).convert('L')
    image = transformation(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(F.softmax(output, dim=1), 1).item()

    classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    return classes[predicted_class]

def classify_emotion(face_image):
    """
    Classify emotion using the above classifier.
    Only consider negative emotions; otherwise return Neutral.
    """
    emotion = classify_image(face_image)
    return emotion if emotion in ["Sad", "Fear", "Disgust", "Angry"] else "Neutral"

# ---------------------------
# Report Writing Function
# ---------------------------
def write_report(video_path, output_video_path, total_frames, male_count, female_count,men,women,
                 lone_women_instances, sos_count, track_license_plates,report_path):
    """Generates and saves a detailed report of the video analysis."""
    # Compute gender ratio (M/F)
    if female_count == 0:
        gender_ratio = "Only males detected"
    elif male_count == 0:  # Handle the case where male_count is 0
        gender_ratio = "Only females detected"
    else:
        gender_ratio = f"{men // women}:1" if male_count >= female_count else f"1:{women // men}"

    # Compute unique number plates
    unique_plates = len(set(track_license_plates.values()))
    avg_plates_per_frame = unique_plates / total_frames if total_frames else 0

    analysis_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    report_dir = os.path.dirname(report_path)  # Get the directory part of the path
    if not os.path.exists(report_dir):
        os.makedirs(report_dir)

    # report_path = "NEW_NEW_report.txt"
    with open(report_path, "w") as f:
        f.write("Video Analysis Report\n")
        f.write("===================\n\n")
        f.write(f"Analysis performed on: {analysis_time}\n")
        f.write(f"Input video: {video_path}\n")
        f.write(f"Output video: {output_video_path}\n\n")
        f.write("Statistics:\n")
        f.write(f"- Total frames processed: {total_frames}\n")
        # f.write(f"- Total males detected: {male_count}\n")
        # f.write(f"- Total females detected: {female_count}\n")
        f.write(f"- Finally Total Men detected: {men}\n")
        f.write(f"- Finally Total Women detected: {women}\n")
        f.write(f"- Gender ratio (M/F): {gender_ratio}\n")
        f.write(f"- Lone women detected (tracking IDs): {len(lone_women_instances)}\n")
        f.write(f"- Total SOS instances detected: {sos_count}\n")
        f.write(f"- Unique number plates: {unique_plates}\n")
        f.write(f"- Average plates per frame: {avg_plates_per_frame:.2f}\n\n")
        if lone_women_instances:
            f.write("Lone Women Detected (tracking IDs):\n")
            for tid in lone_women_instances:
                f.write(f"- ID: {tid}\n")
    print(f"Detection completed. Report saved to `{report_path}`.")

# ---------------------------
# ANPR Function
# ---------------------------

# Load license plate detection model


def read_license_plate(license_plate_crop):
    """Reads the license plate text using EasyOCR."""
    detections = reader.readtext(license_plate_crop)
    for detection in detections:
        _, text, score = detection
        text = text.upper().replace(' ', '')
        return text if len(text) > 3 else None  # Avoid false positives
    return None

def perform_anpr(frame, car_bbox):
    """
    Detects the license plate inside the car and extracts the number.
    """
    x1, y1, x2, y2 = car_bbox
    car_crop = frame[y1:y2, x1:x2]

    if car_crop.size == 0 or car_crop.shape[0] == 0 or car_crop.shape[1] == 0:
        return None  # Skip ANPR if the crop is invalid

    license_plates = license_plate_detector(car_crop)[0]
    for license_plate in license_plates.boxes.data.tolist():
        lx1, ly1, lx2, ly2, score, _ = license_plate
        lx1, ly1, lx2, ly2 = map(int, [lx1, ly1, lx2, ly2])
        license_plate_crop = car_crop[ly1:ly2, lx1:lx2]
        license_text = read_license_plate(license_plate_crop)
        if license_text:
            return license_text
    return None


# Lone Women Tracking


def update_lone_women(detected_women, current_timestamp):
    """
    Update the tracking of lone women.
    A woman is confirmed as "lone" if she remains the only detected woman for at least LONE_WOMAN_THRESHOLD seconds.
    """
    global lone_women_tracker
    active_ids = set(lone_women_tracker.keys())
    detected_set = set(detected_women)

    # For IDs no longer detected, check if they reached the threshold
    for woman_id in list(active_ids - detected_set):
        if lone_women_tracker[woman_id]['duration'] >= LONE_WOMAN_THRESHOLD:
            lone_women_tracker[woman_id]['confirmed'] = True
        else:
            del lone_women_tracker[woman_id]

    # Update or add currently detected women
    for woman_id in detected_set:
        if woman_id not in lone_women_tracker:
            lone_women_tracker[woman_id] = {'start_time': current_timestamp, 'duration': 0, 'confirmed': False}
        else:
            lone_women_tracker[woman_id]['duration'] = current_timestamp - lone_women_tracker[woman_id]['start_time']

# ---------------------------
# SOS Detection using Mediapipe
# ---------------------------
if not os.path.exists("SOS_Logs"):
    os.makedirs("SOS_Logs")

def log_sos_event(frame):
    """Saves an SOS frame and logs the timestamp."""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"SOS_Logs/SOS_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"[ALERT] SOS Detected! Frame saved: {filename}")
    with open("SOS_Logs/sos_log.txt", "a") as log_file:
        log_file.write(f"{timestamp} - SOS Detected!\n")

import mediapipe as mp
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

person_motion = deque(maxlen=10)  # For tracking movement

def detect_movement(pose_landmarks, frame_width, frame_height):
    """Detects running and sudden movements."""
    global person_motion
    key_points = [pose_landmarks.landmark[i] for i in [mp_pose.PoseLandmark.LEFT_HIP,
                                                       mp_pose.PoseLandmark.RIGHT_HIP,
                                                       mp_pose.PoseLandmark.LEFT_KNEE,
                                                       mp_pose.PoseLandmark.RIGHT_KNEE]]
    key_positions = [(int(p.x * frame_width), int(p.y * frame_height)) for p in key_points]
    avg_x = sum(p[0] for p in key_positions) // len(key_positions)
    avg_y = sum(p[1] for p in key_positions) // len(key_positions)
    person_motion.append((avg_x, avg_y))
    if len(person_motion) > 5:
        deltas = np.diff(person_motion, axis=0)
        speeds = np.sum(np.abs(deltas), axis=0)
        if speeds[1] > 50:
            return "Running"
        if np.linalg.norm(deltas[-1]) > 40:
            return "Sudden Movement"
    return "Normal"

def detect_hand_gesture(hand_landmarks, frame_width, frame_height):
    """Detects raised hand gesture."""
    landmarks = [hand_landmarks.landmark[i] for i in range(21)]
    y_positions = [l.y for l in landmarks]
    avg_y = sum(y_positions) / len(y_positions)
    hand_height = avg_y * frame_height
    if hand_height < frame_height // 3:
        return "Raised Hand"
    return "Normal"

SOS_cooldown = {}  # To prevent multiple triggers for the same track
SOS_THRESHOLD = 3   # Number of consecutive frames needed to trigger SOS

def perform_sos(person, track_id, frame_number):
    """
    Detects SOS condition based on facial emotion, body movement, and hand gesture.
    Returns "SOS Detected" if at least two of the conditions are triggered; otherwise "Safe".
    """
    global SOS_cooldown
    if track_id in SOS_cooldown and frame_number - SOS_cooldown[track_id] < SOS_THRESHOLD*2:
        return "Safe"  # within cooldown period

    # Check for valid crop; skip if it's too small
    min_dim = 20  # set a minimum acceptable dimension
    if person is None or person.size == 0 or person.shape[0] < min_dim or person.shape[1] < min_dim:
        # Optionally, you could log this once per track if needed.
        return "Safe"

    rgb_person = cv2.cvtColor(person, cv2.COLOR_BGR2RGB)
    pose_results = pose.process(rgb_person)
    hand_results = hands.process(rgb_person)

    emotion = classify_emotion(person)
    movement = "Normal"
    hand_gesture = "Normal"

    if pose_results.pose_landmarks:
        movement = detect_movement(pose_results.pose_landmarks, person.shape[1], person.shape[0])
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            hand_gesture = detect_hand_gesture(hand_landmarks, person.shape[1], person.shape[0])

    triggers = sum([
        emotion in ["Sad", "Fear", "Disgust", "Angry"],
        movement in ["Running", "Sudden Movement"],
        hand_gesture == "Raised Hand"
    ])

    if triggers >= 2 and frame_number - SOS_cooldown.get(track_id, 0) > SOS_THRESHOLD * 2:
        SOS_cooldown[track_id] = frame_number
        log_sos_event(person)
        return "SOS Detected"
    return "Safe"


def shrink_bbox(x1, y1, x2, y2, shrink_factor=0.2):
    """
    Shrinks the bounding box by the given shrink_factor (default 20%).
    Returns new coordinates.
    """
    w = x2 - x1
    h = y2 - y1
    dx = int(w * shrink_factor / 2)
    dy = int(h * shrink_factor / 2)
    return x1 + dx, y1 + dy, x2 - dx, y2 - dy


# if __name__ == "__main__":
#     detect_objects("/content/sample_video_ppl.mp4", "finally_final_output_video.mp4")

# if __name__ == "__main__":
#     detect_objects("/content/sample_vide_ppl.mp4", "output_video.mp4","1/Report.txt","1/Car_results.csv")


In [48]:
def detect_objects(video_path, output_video_path,report_path,csv_path, conf=0.5):
    """
    Detects objects in a video, classifies gender,
    performs SOS detection, tracks lone women, and processes ANPR for cars.
    Draws bounding boxes: blue for Men, green for Women, red for SOS.
    Generates output video, report, CSV, and SOS logs.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video stream or file")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or 1
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    track_gender = {}
    track_license_plates = {}  # For cars
    cars_data = []  # To be saved as CSV
    male_count, female_count, sos_count = 0, 0, 0

    # Process each frame
    with tqdm(total=total_frames, desc="Processing Video", unit="frame") as pbar:
        frame_number = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_number += 1
            current_timestamp = frame_number / fps  # in seconds
            detected_women = []  # For lone women tracking in this frame

            # Run YOLO detection
            yolo_results = model.predict(source=frame, conf=conf, verbose=False)
            detections = []
            for result in yolo_results:
                boxes = result.boxes.cpu().numpy()
                for box in boxes:
                    b = box.xyxy[0].astype(int)
                    class_id = int(box.cls[0])
                    confidence = box.conf[0]
                    if confidence > conf and model.names[class_id] in ["person", "car"]:
                        detections.append([[b[0], b[1], b[2], b[3]], confidence, class_id])

            # Update tracks with DeepSORT
            tracks = tracker.update_tracks(detections, frame=frame)

            # Process each track
            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                x1, y1, x2, y2 = map(int, track.to_ltrb())
                class_id = track.det_class

                if model.names[class_id] == "person":
                    x1, y1, x2, y2 = map(int, track.to_ltrb())
                    x1 = max(0, x1)
                    y1 = max(0, y1)
                    x2 = min(frame.shape[1], x2)
                    y2 = min(frame.shape[0], y2)
                    if x2 <= x1 or y2 <= y1:
                        continue

                    cropped_person = frame[y1:y2, x1:x2]
                    if cropped_person.size == 0 or cropped_person.shape[0] < 20 or cropped_person.shape[1] < 20:
                        continue # Skip SOS detection if the crop is too small\n
                    if track_id not in track_gender:
                        if cropped_person.shape[0] > 0 and cropped_person.shape[1] > 0:
                            gender = predict_gender(cropped_person)
                            track_gender[track_id] = gender
                            if gender == "Man":
                                male_count += 1
                            elif gender == "Woman":
                                female_count += 1
                        else:
                            track_gender[track_id] = "Unknown"
                            gender = "Unknown"
                    else:
                        gender = track_gender[track_id]

                    print(f"Frame {frame_number}: Track ID {track_id}, Gender {gender}")

                    if gender == "Man":
                        male_count += 1
                        # Draw blue bounding box for Men (BGR: Blue)
                        new_x1, new_y1, new_x2, new_y2 = shrink_bbox(x1, y1, x2, y2, shrink_factor=0.2)
                        cv2.rectangle(frame, (new_x1, new_y1), (new_x2, new_y2), (255, 0, 0), 2)
                        cv2.putText(frame, "Man", (new_x1, new_y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                    elif gender == "Woman":
                        female_count += 1
                        detected_women.append(track_id)
                        # Run face detection on the cropped person for further emotion analysis
                        if cropped_person.shape[0] > 0 and cropped_person.shape[1] > 0:
                            face_results = face_model.predict(source=cropped_person, conf=0.5, verbose=False)
                            if face_results and len(face_results[0].boxes) > 0:
                                # Optionally, use face detection to crop face region (if needed by your pipeline)
                                face_box = face_results[0].boxes[0].xyxy[0].int().tolist()
                                x1f, y1f, x2f, y2f = face_box
                                cropped_face = cropped_person[y1f:y2f, x1f:x2f]
                                # You can also use classify_emotion(cropped_face) here if desired.
                        # Check for SOS condition
                        sos_status = perform_sos(cropped_person, track_id, frame_number)
                        new_x1, new_y1, new_x2, new_y2 = shrink_bbox(x1, y1, x2, y2, shrink_factor=0.2)
                        if sos_status == "SOS Detected":
                            sos_count += 1
                            # Draw red bounding box for SOS
                            cv2.rectangle(frame, (new_x1, new_y1), (new_x2, new_y2), (0, 0, 255), 2)
                            cv2.putText(frame, "SOS Detected", (new_x1, new_y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        else:
                            # Draw green bounding box for Woman
                            cv2.rectangle(frame, (new_x1, new_y1), (new_x2, new_y2), (0, 255, 0), 2)
                            cv2.putText(frame, "Woman", (new_x1, new_y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                elif model.names[class_id] == "car":
                    if track_id not in track_license_plates:
                        license_plate_text = perform_anpr(frame, (x1, y1, x2, y2))
                        if license_plate_text:
                            track_license_plates[track_id] = license_plate_text
                            cars_data.append(["Car", track_id, None, license_plate_text])
            # Update lone women tracking based on detected women IDs in this frame
            update_lone_women(detected_women, current_timestamp)
            out.write(frame)
            pbar.update(1)

    cap.release()
    out.release()
    # mencounted = sum(1 for gender in track_gender.values() if gender == "Man")
    # womencounted = sum(1 for gender in track_gender.values() if gender == "Woman")
    men = sum(1 for gender in track_gender.values() if gender == "Man")
    women = sum(1 for gender in track_gender.values() if gender == "Woman")
    confirmed_lone_women = [k for k, v in lone_women_tracker.items() if v['confirmed']]
    write_report(video_path, output_video_path, total_frames, male_count, female_count,men,women,
                 confirmed_lone_women, sos_count, track_license_plates,report_path)

    # Save car ANPR results to CSV
    df = pd.DataFrame(cars_data, columns=["Object", "ID", "Gender", "License Plate"])
    df.to_csv(csv_path, index=False)
    print("Detection completed. Report, CSV, and output video have been generated.")



In [49]:
tracker = DeepSort(max_age=120, n_init=3, nn_budget=10)  # Adjusted max_age
if __name__ == "__main__":
    detect_objects("/content/drive/MyDrive/carwomen.mp4", "car_women2_output.mp4","2/car_women2_Report.txt","2/car_women2_results.csv")


Processing Video:   0%|          | 1/733 [00:00<03:30,  3.48frame/s]

Frame 3: Track ID 1, Gender Woman


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4969: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Processing Video:   1%|          | 4/733 [00:01<03:20,  3.63frame/s]

Frame 4: Track ID 1, Gender Woman
Frame 5: Track ID 1, Gender Woman


Processing Video:   1%|          | 6/733 [00:01<02:53,  4.20frame/s]

...

Frame 732: Track ID 7, Gender Woman

0: 640x576 (no detections), 11.6ms
Speed: 5.1ms preprocess, 11.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 576)


Processing Video: 100%|█████████▉| 732/733 [02:21<00:00,  6.69frame/s]

Frame 733: Track ID 7, Gender Woman

0: 640x608 (no detections), 10.5ms
Speed: 5.2ms preprocess, 10.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 608)


Processing Video: 100%|██████████| 733/733 [02:21<00:00,  5.17frame/s]

Detection completed. Report saved to `2/car_women2_Report.txt`.
Detection completed. Report, CSV, and output video have been generated.


In [53]:
# import os
# import shutil
# source_file = "/content/2/kidnap1_results.csv"
# shutil.move(source_file,"/content/3")


'/content/3/kidnap1_results.csv'

In [54]:
import os
import shutil

# Define the source file and destination folder
source_file = "car_women2_output.mp4"
destination_folder = "2"

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Move the file into the destination folder
shutil.move(source_file, os.path.join(destination_folder, source_file))

# Create a zip archive of the folder
shutil.make_archive(destination_folder, 'zip', destination_folder)
print(f"Moved {source_file} to {destination_folder} and zipped {destination_folder} folder")


Moved car_women2_output.mp4 to 2 and zipped 2 folder


In [36]:
tracker = DeepSort(max_age=120, n_init=3, nn_budget=10)  # Adjusted max_age
if __name__ == "__main__":
    detect_objects("/content/kidnap1.mp4", "kidnap1_output.mp4","3/kidnap1_Report.txt","2/kidnap1_results.csv")


Processing Video:   1%|          | 2/342 [00:00<00:27, 12.47frame/s]

Frame 3: Track ID 1, Gender Woman
Frame 3: Track ID 2, Gender Woman

0: 640x416 (no detections), 14.4ms
Speed: 3.1ms preprocess, 14.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 416)

0: 544x640 (no detections), 20.4ms
Speed: 3.9ms preprocess, 20.4ms inference, 0.9ms postprocess per image at shape (1, 3, 544, 640)

0: 480x640 1 license_plate, 16.6ms
Speed: 3.6ms preprocess, 16.6ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 (no detections), 16.6ms
Speed: 3.6ms preprocess, 16.6ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)

0: 640x544 (no detections), 15.9ms
Speed: 3.9ms preprocess, 15.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 544)


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4969: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Frame 3: Track ID 8, Gender Man
Frame 4: Track ID 1, Gender Woman
Frame 4: Track ID 2, Gender Woman

0: 640x416 (no detections), 14.2ms
Speed: 4.5ms preprocess, 14.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 416)

0: 544x640 (no detections), 14.1ms
Speed: 3.7ms preprocess, 14.1ms inference, 0.9ms postprocess per image at shape (1, 3, 544, 640)

0: 480x640 (no detections), 13.5ms
Speed: 3.9ms preprocess, 13.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 (no detections), 13.6ms
Speed: 3.8ms preprocess, 13.6ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 640x544 1 license_plate, 14.7ms
Speed: 4.1ms preprocess, 14.7ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 544)


Processing Video:   1%|          | 4/342 [00:01<01:50,  3.06frame/s]

...

Frame 341: Track ID 1, Gender Woman
Frame 341: Track ID 4, Gender Woman

0: 640x480 (no detections), 20.2ms
Speed: 6.2ms preprocess, 20.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

0: 64x640 (no detections), 23.5ms
Speed: 0.8ms preprocess, 23.5ms inference, 0.9ms postprocess per image at shape (1, 3, 64, 640)

0: 640x448 (no detections), 21.7ms
Speed: 3.4ms preprocess, 21.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 448)

0: 640x192 (no detections), 21.0ms
Speed: 1.6ms preprocess, 21.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 192)


Processing Video: 100%|█████████▉| 341/342 [01:53<00:00,  3.97frame/s]

Frame 342: Track ID 1, Gender Woman
Frame 342: Track ID 4, Gender Woman

0: 640x480 (no detections), 10.4ms
Speed: 4.3ms preprocess, 10.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

0: 64x640 (no detections), 14.2ms
Speed: 0.8ms preprocess, 14.2ms inference, 0.7ms postprocess per image at shape (1, 3, 64, 640)

0: 640x448 (no detections), 11.5ms
Speed: 5.3ms preprocess, 11.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 448)

0: 640x192 (no detections), 13.9ms
Speed: 1.6ms preprocess, 13.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 192)


Processing Video: 100%|██████████| 342/342 [01:53<00:00,  3.01frame/s]

Detection completed. Report saved to `3/kidnap1_Report.txt`.
Detection completed. Report, CSV, and output video have been generated.


In [55]:

# Define the source file and destination folder
source_file = "kidnap1_output.mp4"
destination_folder = "3"

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Move the file into the destination folder
shutil.move(source_file, os.path.join(destination_folder, source_file))

# Create a zip archive of the folder
shutil.make_archive(destination_folder, 'zip', destination_folder)
print(f"Moved {source_file} to {destination_folder} and zipped {destination_folder} folder")


Moved kidnap1_output.mp4 to 3 and zipped 3 folder


In [41]:
tracker = DeepSort(max_age=120, n_init=3, nn_budget=10)  # Adjusted max_age
if __name__ == "__main__":
    detect_objects("/content/car_kidnap.mp4", "car_kidnap_output.mp4","4/car_kidnap_Report.txt","4/car_kidnap_results.csv")


Processing Video:   0%|          | 1/681 [00:00<01:10,  9.63frame/s]

Frame 3: Track ID 1, Gender Woman

0: 640x480 (no detections), 11.1ms
Speed: 4.4ms preprocess, 11.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)


Processing Video:   0%|          | 3/681 [00:00<01:17,  8.71frame/s]

Frame 4: Track ID 1, Gender Woman

0: 640x480 (no detections), 9.7ms
Speed: 4.3ms preprocess, 9.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)


Processing Video:   1%|          | 4/681 [00:00<01:19,  8.46frame/s]

...

Frame 680: Track ID 32, Gender Man

0: 640x224 (no detections), 15.3ms
Speed: 2.3ms preprocess, 15.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 224)


Processing Video: 100%|█████████▉| 680/681 [01:31<00:00,  7.20frame/s]

Frame 680: Track ID 42, Gender Man
Frame 680: Track ID 44, Gender Man
Frame 681: Track ID 32, Gender Man

0: 640x224 (no detections), 17.7ms
Speed: 4.5ms preprocess, 17.7ms inference, 5.2ms postprocess per image at shape (1, 3, 640, 224)


Processing Video: 100%|██████████| 681/681 [01:31<00:00,  7.44frame/s]

Frame 681: Track ID 42, Gender Man
Frame 681: Track ID 44, Gender Man
Detection completed. Report saved to `4/car_kidnap_Report.txt`.
Detection completed. Report, CSV, and output video have been generated.


In [56]:

# Define the source file and destination folder
source_file = "car_kidnap_output.mp4"
destination_folder = "4"

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Move the file into the destination folder
shutil.move(source_file, os.path.join(destination_folder, source_file))

# Create a zip archive of the folder
shutil.make_archive(destination_folder, 'zip', destination_folder)
print(f"Moved {source_file} to {destination_folder} and zipped {destination_folder} folder")


Moved car_kidnap_output.mp4 to 4 and zipped 4 folder


In [52]:
tracker = DeepSort(max_age=120, n_init=3, nn_budget=10)  # Adjusted max_age
if __name__ == "__main__":
    detect_objects("/content/safewomen.mp4", "safewomen_output.mp4","5/safewomen_Report.txt","5/safewomen_results.csv")


Processing Video:   1%|          | 2/240 [00:00<01:16,  3.11frame/s]

Frame 3: Track ID 1, Gender Man
Frame 3: Track ID 2, Gender Woman
Frame 3: Track ID 3, Gender Man


Processing Video:   2%|▏         | 4/240 [00:01<01:32,  2.55frame/s]

Frame 4: Track ID 1, Gender Man
Frame 4: Track ID 2, Gender Woman
Frame 4: Track ID 3, Gender Man
Frame 5: Track ID 1, Gender Man
Frame 5: Track ID 2, Gender Woman
Frame 5: Track ID 3, Gender Man


Processing Video:   2%|▎         | 6/240 [00:02<01:36,  2.43frame/s]

...

Frame 239: Track ID 1, Gender Man

0: 640x192 (no detections), 12.3ms
Speed: 2.1ms preprocess, 12.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 192)
Frame 239: Track ID 3, Gender Man

0: 640x96 (no detections), 12.1ms
Speed: 1.5ms preprocess, 12.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 96)

0: 640x384 (no detections), 11.6ms
Speed: 3.8ms preprocess, 11.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x352 (no detections), 11.5ms
Speed: 6.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 352)

0: 640x64 (no detections), 11.7ms
Speed: 1.2ms preprocess, 11.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 64)


Processing Video: 100%|█████████▉| 239/240 [01:18<00:00,  2.63frame/s]

Frame 240: Track ID 1, Gender Man

0: 640x192 (no detections), 19.6ms
Speed: 3.3ms preprocess, 19.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 192)

0: 640x96 (no detections), 25.3ms
Speed: 3.8ms preprocess, 25.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 96)

0: 640x384 (no detections), 22.0ms
Speed: 5.6ms preprocess, 22.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x352 (no detections), 11.2ms
Speed: 4.1ms preprocess, 11.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 352)

0: 640x64 (no detections), 19.3ms
Speed: 1.3ms preprocess, 19.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 64)


Processing Video: 100%|██████████| 240/240 [01:19<00:00,  3.03frame/s]

Detection completed. Report saved to `5/safewomen_Report.txt`.
Detection completed. Report, CSV, and output video have been generated.


In [57]:

# Define the source file and destination folder
source_file = "safewomen_output.mp4"
destination_folder = "5"

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Move the file into the destination folder
shutil.move(source_file, os.path.join(destination_folder, source_file))

# Create a zip archive of the folder
shutil.make_archive(destination_folder, 'zip', destination_folder)
print(f"Moved {source_file} to {destination_folder} and zipped {destination_folder} folder")


Moved safewomen_output.mp4 to 5 and zipped 5 folder


In [58]:
shutil.make_archive("/content/SOS_Logs", 'zip', "/content/SOS_Logs")

'/content/SOS_Logs.zip'